## Importing Libraries

In [1]:
import os
from dotenv import load_dotenv

# pytorch
import torch

# huggingface
from huggingface_hub import login
from transformers import T5EncoderModel, BitsAndBytesConfig

# diffusers
from diffusers import StableDiffusion3Pipeline

/home/pathfinder/anaconda3/envs/diffusers/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/pathfinder/anaconda3/envs/diffusers/lib/python3.11/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


## Login to Hugging Face

In [2]:
load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token,  # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [4]:
################################################################################
# Quantization parameters
################################################################################
load_in_4bit = True
bnb_4bit_compute_dtype = torch.float16

################################################################################
# Generation parameters
################################################################################
#n_steps = 40
#high_noise_frac = 0.8
num_inference_steps = 30
guidance_scale = 100.0

## Model

In [5]:
"""
# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
)
base.to("cuda")
base.unet = torch.compile(base.unet, mode="reduce-overhead", fullgraph=True)
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")
refiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)
"""

'\n# load both base & refiner\nbase = DiffusionPipeline.from_pretrained(\n    "stabilityai/stable-diffusion-xl-base-1.0",\n    torch_dtype=torch.float16,\n    variant="fp16",\n    use_safetensors=True\n)\nbase.to("cuda")\nbase.unet = torch.compile(base.unet, mode="reduce-overhead", fullgraph=True)\nrefiner = DiffusionPipeline.from_pretrained(\n    "stabilityai/stable-diffusion-xl-refiner-1.0",\n    text_encoder_2=base.text_encoder_2,\n    vae=base.vae,\n    torch_dtype=torch.float16,\n    use_safetensors=True,\n    variant="fp16",\n)\nrefiner.to("cuda")\nrefiner.unet = torch.compile(refiner.unet, mode="reduce-overhead", fullgraph=True)\n'

In [ ]:
# load stable-diffusion-3-medium-diffusers
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3-medium-diffusers",
    device_map="balanced",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

model_index.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

text_encoder_2/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

...

tokenizer_3/tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

transformer/config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

## Inference

In [ ]:
prompt = "A majestic lion jumping from a big stone at night"

In [ ]:
"""
# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]
"""

In [ ]:
image = pipe(
    prompt,
    negative_prompt="",
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale
).images[0]

In [ ]:
image

In [ ]:
# save image
image.save("images/output.png")